In [1]:
import pyspark as sp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,date_format,concat,substring,lit,lpad,to_timestamp,when
import  pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()

24/04/17 23:52:33 WARN Utils: Your hostname, Subins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.65.210.91 instead (on interface en0)
24/04/17 23:52:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/17 23:52:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/17 23:52:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/17 23:52:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df=spark.read.option("header","true").option("inferSchema", "true").csv("./dataConverted/2018/2018_039620.csv")

In [16]:
df.show(10000)

+-----+-----+----------+-----+------+------+---+-------+-----+-------+-------+------------+--------+-------------------+-----+------+
| USAF| WBAN|      date| time|   lat|  long|ele|winddir|  sky|visdist|airtemp|dewpointtemp|atmpress|          timestamp|month|season|
+-----+-----+----------+-----+------+------+---+-------+-----+-------+-------+------------+--------+-------------------+-----+------+
|39620|99999|2018-01-01|00:00|  52.7|-8.917| 20|    240|22000|  20000|    6.9|         3.5|   993.5|2018-01-01 00:00:00|    1|Winter|
|39620|99999|2018-01-01|00:00|52.702|-8.925| 14|    240| 2743|   9999|    7.0|         4.0|  9999.9|2018-01-01 00:00:00|    1|Winter|
|39620|99999|2018-01-01|00:30|52.702|-8.925| 14|    240|99999|   9999|    7.0|         4.0|  9999.9|2018-01-01 00:30:00|    1|Winter|
|39620|99999|2018-01-01|01:00|  52.7|-8.917| 20|    230| 6000|  16000|    7.2|         4.3|   993.3|2018-01-01 01:00:00|    1|Winter|
|39620|99999|2018-01-01|01:00|52.702|-8.925| 14|    230| 1372|

#---------------------Pre-Processing-------------------------------------#

In [5]:
df = df.withColumn("lat", df["lat"] / 1000)


In [6]:
df = df.withColumn("long", df["long"] / 1000)

In [7]:
df = df.withColumn("airtemp", df["airtemp"] / 10)

In [8]:
df = df.withColumn("dewpointtemp", df["dewpointtemp"] / 10)

In [9]:
df = df.withColumn("atmpress", df["atmpress"] / 10)

In [10]:
df = df.withColumn("date", 
                   concat(substring(col("date"), 1, 4), 
                          lit("-"), 
                          substring(col("date"), 5, 2), 
                          lit("-"), 
                          substring(col("date"), 7, 2)))

In [11]:
df = df.withColumn("time", lpad(col("time"), 4, "0"))

In [12]:
df = df.withColumn("time", 
                   concat(substring(col("time"), 1, 2), 
                          lit(":"),
                          substring(col("time"), 3, 2)))

In [13]:
df = df.withColumn("timestamp", to_timestamp(concat(col("date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm"))

In [14]:
def get_season(month):
    if 3 <= month <= 5:
        return "Spring"
    elif 6 <= month <= 8:
        return "Summer"
    elif 9 <= month <= 11:
        return "Autumn"
    else:
        return "Winter"

In [15]:
df = df.withColumn("month", col("date").substr(6, 2).cast("int"))
df = df.withColumn("season", when((col("month") >= 3) & (col("month") <= 5), "Spring")
                              .when((col("month") >= 6) & (col("month") <= 8), "Summer")
                              .when((col("month") >= 9) & (col("month") <= 11), "Autumn")
                              .otherwise("Winter"))